In [19]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from dotenv import load_dotenv

load_dotenv()
# Set your OpenRouter API key
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
openrouter_model = os.getenv("OPENROUTER_MODEL", "openai/gpt-4o")
# Define your OpenRouter LLM
llm = ChatOpenAI(
    model_name=openrouter_model,  # Replace with a model that supports tool calling on OpenRouter
    openai_api_key=openrouter_api_key,
    base_url="https://openrouter.ai/api/v1"
)

# Define your tools
@tool
def get_current_weather(location: str):
    """Get the current weather in a given location."""
    if location.lower() == "hanoi":
        return "The weather in Hanoi is 25 degrees Celsius and sunny."
    else:
        return "Weather information not available for this location."

tools = [get_current_weather]

# Bind the tools to the LLM
llm_with_tools = llm.bind_tools(tools)

# Define the LangGraph state
from typing import TypedDict, List, Dict
from langchain_core.messages import BaseMessage

class GraphState(TypedDict):
    messages: List[BaseMessage]
    tool_calls: Dict

# Define the LangGraph nodes
def call_model(state: GraphState):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response], "tool_calls": {}}

def call_tool(state: GraphState):
    message = state["messages"][-1]
    tool_call = message.tool_calls[0]
    tool_name = tool_call.name
    tool_args = tool_call.args
    if tool_name == "get_current_weather":
        observation = get_current_weather(location=tool_args["location"])
        return {"messages": [observation], "tool_calls": {}}
    else:
        return {"messages": [BaseMessage(content="Unknown tool")], "tool_calls": {}}

def should_call_tool(state: GraphState):
    message = state["messages"][-1]
    if message.tool_calls:
        return "call_tool"
    else:
        return "call_model"

# Build the LangGraph
graph_builder = StateGraph(GraphState)
graph_builder.add_node("model", call_model)
graph_builder.add_node("call_tool", call_tool)
graph_builder.add_conditional_edges("model", should_call_tool, {"call_tool": "call_tool", "call_model": END})
graph_builder.add_edge("call_tool", "model")
graph_builder.set_entry_point("model")
graph = graph_builder.compile()

# Run the LangGraph
inputs = {"messages": [{"role": "user", "content": "What's the weather in Hanoi?"}]}
result = graph.invoke(inputs)
print(result)

NotFoundError: Error code: 404 - {'error': {'message': 'No endpoints found that support tool use. To learn more about provider routing, visit: https://openrouter.ai/docs/provider-routing', 'code': 404}, 'user_id': 'user_2td1OKkKSWQggO71c9GJB9Ao5tM'}